In [52]:
# Importing packages
from bs4 import BeautifulSoup
import requests, re, time, random, csv
import pandas as pd

In [53]:
# Settings for headers:
headers = {
    'Connection': 'keep-alive',
    'Access-Control-Request-Headers': 'content-type',
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

In [9]:
# Grabbing NYC zips. I am excluding those I've already run: 10011, 10012, 10013, 10026, 10033, 10128, 11205
myzips = ['10001','10002','10003','10004','10005','10006','10007','10009','10010',                 \
                  '10014','10016','10017','10018','10019','10020','10021','10022','10023','10024', \
          '10025',        '10027','10028','10029','10030','10031','10032',        '10034','10035', \
          '10036','10037','10038','10039','10040','10044','10045','10065','10115','10119',         \
          '10154','10278','10280','10301','10302','10303','10304','10305','10306','10307','10309', \
          '10310','10312','10314','10451','10452','10453','10454','10455','10456','10457','10458', \
          '10459','10460','10461','10462','10463','10464','10465','10466','10467','10468','10469', \
          '10471','10472','10473','10474','10475','10514','10543','10553','10573','10701','10705', \
          '10911','10965','10977','11001','11021','11050','11101','11102','11103','11104','11105', \
          '11106','11111','11112','11201','11202','11203','11204',        '11206','11207','11208', \
          '11209','11210','11211','11212','11213','11214','11215','11216','11217','11218','11219', \
          '11220','11221','11222','11223','11224','11225','11226','11228','11229','11230','11231', \
          '11232','11233','11234','11235','11236','11237','11238','11239','11252','11354','11355', \
          '11356','11357','11358','11360','11361','11362','11364','11365','11366','11367','11368', \
          '11369','11370','11371','11372','11373','11374','11375','11377','11378','11379','11385', \
          '11411','11412','11413','11414','11415','11416','11417','11418','11419','11420','11421', \
          '11422','11423','11424','11425','11426','11427','11428','11429','11430','11431','11432', \
          '11433','11434','11435','11436','11439','11451','11471','11510','11548','11566','11577', \
          '11580','11598','11629','11691','11692','11693','11694','11695','11731','11798','11968', \
          '12423','12428','12435','12458','12466','12473','12528','12701','12733','12734','12737', \
          '12750','12751','12754','12758','12759','12763','12764','12768','12779','12783','12786', \
          '12788','12789','13731','16091','20459']
len(myzips)

229

In [54]:
myzips = ['12754']
len(myzips)

1

In [27]:
len(myzips)

1

In [55]:
# Actual web scraping - scraping Psychology Today for these zipcodes:
# But first I need to run the function I've defined (see below)
from single_url_scrape import scrape_single_page
for myzip in myzips:
    
    print('Starting zipcode ' + myzip)
    # Defining urls we are starting with and will be using
    starturl = 'https://therapists.psychologytoday.com/rms/prof_results.php?search=' + myzip
    longurl = 'https://therapists.psychologytoday.com/rms/prof_results.php?search=' + myzip + '&rec_next='
    
    # Beautifulsoup of the start url
    response_zip = requests.get(starturl, headers=headers).text
    soup_zip = BeautifulSoup(response_zip, 'html.parser')

    # Manually construct the list of urls for ALL available search result pages in that tip - 20 therapists per page:
    pages_url_list = []
    i = 1
    while True:
        myurl = longurl + str(i)
        response = requests.get(myurl, headers=headers).text
        soup = BeautifulSoup(response, 'html.parser')
        alert = soup.find('div', attrs={'class': 'alert-alert'})
        if alert is None:
            i = i + 20
            pages_url_list.append(myurl)
            time.sleep(random.randint(0,1))
        else:
            break

    print('Number of group urls for this zip is: ' + str(len(pages_url_list)))
    if len(pages_url_list) == 0:
        continue

    # Loop through all urls found above (for pages of therapist search for that zip) and 
    # create a list of urls - one for each psychotherapist:

    therapist_urls = []
    for url in pages_url_list:
        response = requests.get(url, headers = headers).text
        soup = BeautifulSoup(response, 'html.parser')
        therapists = soup.find_all('a', {'class': "result-name"})
        for i in range(len(therapists)):
            therapist_urls.append(therapists[i]["href"])
            time.sleep(random.randint(0,1))
    print('Total number of therapists in this zip is: ' + str(len(therapist_urls)))
    
    # Now that we have the list of individual therapists' urls, we can scrape that zipcode:

    bigdf = pd.DataFrame()
    for index, url in enumerate(therapist_urls):
        response_one = requests.get(url, headers = headers).text
        one_therap = BeautifulSoup(response_one, 'html.parser')
        if one_therap.find('h1', {'itemprop': 'name'}) == None:
            continue
        smalldf = scrape_single_page(one_therap, myzip)
        bigdf = bigdf.append(smalldf)
        # Random sleep to avoid getting banned from the server
        time.sleep(random.randint(0, 1))
        # Log the progress
        print('Finished url ' + str(index + 1))

    # Drop duplicate rows in case they are there:
    bigdf = bigdf.drop_duplicates()
    # Save big data frame as a csv file
    bigdf.to_csv(("zip_" + myzip + ".csv"))

Starting zipcode 12754
Number of group urls for this zip is: 1
Total number of therapists in this zip is: 15
Finished url 1
Finished url 2
Finished url 3
Finished url 4
Finished url 5
Finished url 6
Finished url 7
Finished url 8
Finished url 9
Finished url 10
Finished url 11
Finished url 12
Finished url 13
Finished url 14
Finished url 15


In [ ]:
# placeholder

In [5]:
# This function parses the Beautifulsoup of one single url (therapist)

def scrape_single_page(one_therap, zipcode):
    
    dic = {}
    
    # Therapist name - just for checking purposes:
    therap_name = one_therap.find('h1', {'itemprop': 'name'}).text.strip()
    
    # Write down the zip defined above:
    zipcode = zipcode
    
    # Grab qualifications of one therapist
    qualif_div = one_therap.find('div', {'class': "profile-qualifications"})
    allqualif = qualif_div.find_all('li')   # they are saved in 'li'

    # Collecting Years in Practice:
    years_check = [i.text.split('\n')[2].strip() for i in allqualif if "Years in Practice" in i.text]
    if len(years_check) > 0:
        years_in_practice = years_check[0].replace(' Years', '')
    else:
        years_in_practice = ''
    
    # Collecting School:
    school_check = [i.text.split('\n')[2].strip() for i in allqualif if "School" in i.text]
    if len(school_check) > 0:
        school = school_check[0]
        school = re.sub(r'\([^)]*\)', '', school).strip()
        school = school.replace(', ', '_')
        school = school.replace(',', '_')
        school = school.replace('/ ', '_')
        school = school.replace('/', '_')
        school = school.replace(' - ', '_')
        school = school.replace(': ', '_')
        school = school.replace(':', '_')
        school = school.replace(' :', '_')
        school = school.replace('&', 'and')
        school = school.replace(' ', '_').lower()
        school = 'uni_' + school
    else:
        school = ''
    
    # Collecting Year Graduated:
    yeargrad_check = [i.text.split('\n')[2].strip() for i in allqualif if "Year Graduated" in i.text]
    if len(yeargrad_check) > 0:
        yeargrad = int(yeargrad_check[0])
    else:
        yeargrad = ''
    
    # Collecting Board Certification:
    board_check = [i.text.split('\n')[2].strip() for i in allqualif if "Board Certification" in i.text]
    if len(board_check) > 0:
        board_certif = 1
    else:
        board_certif = 0
    
    # Collecting License number & State:
    license_check = [i.text.split('\n')[2].strip() for i in allqualif if "License No. and State" in i.text]
    if len(license_check) > 0:
        license = license_check[0].split()[0]
        license_state = " ".join(license_check[0].split()[1:])
    else:
        license = ''
        license_state = ''
    
    # Grabbing all his/her titles:
    titles = one_therap.find('div', {'class': "profile-title"}).text.strip()
    titles = titles.replace("\n", "")
    titles = titles.replace('/', '_').strip()
    titles = titles.replace(', ', ',')
    titles = titles.replace('-', '_')
    titles = titles.replace(' ', '_')
    titles = titles.replace(',', ', ')
    
    # Grabbing financial info:
    finances = one_therap.find('div', {'class': "profile-finances"})
    if finances == None:
        per_session = ''
        sliding = ''
        insurance_yes = ''
        payment_methods = 'xNotListed'
    else:
        # Grabbing several financial metrics:
        lis = finances.find_all('li')
        
        # Grab cost per session:
        persession_check = [i.text.split(': ')[1].strip() for i in lis if "Avg Cost (per session)" in i.text]
        if len(persession_check) > 0:
            per_session = persession_check[0].replace('$', '')
            per_session = per_session.replace(' ', '')
        else:
            per_session = ''
            
        # Grab sliding scale (1 = yes, 0 = no):
        sliding_check = [i.text for i in lis if 'Sliding Scale' in i.text]
        if sliding_check != None:
            sliding_check = " ".join(sliding_check)
            if 'Sliding Scale: Yes' in sliding_check:
                sliding = 1
            else:
                sliding = 0
        else:
            sliding = 0
        
        # Grab Accepts Insurance - yes or no:
        insur_check = [i.text.split(': ')[1].strip() for i in lis if "Accepts Insurance" in i.text]
        if len(insur_check) > 0:
            insurance_yes = 1   # insur_check[0]
        else:
            insurance_yes = 0    # ''
        
        # Grab Methods of Payment:
        # payment_check = [i.text for i in lis if "Accepted Payment Methods" in i.text]
        payment_check = [i.text.split('\n')[2].strip().split(',') for i in lis if "Accepted Payment Methods" in i.text]
        if len(payment_check) == 0:
            payment_methods = 'xNotListed'
        else:
            payment_methods = [item for sublist in payment_check for item in sublist]
            for index, item in enumerate(payment_methods):
                out = item.strip().lower()
                out = out.replace(" ", "_")
                out = 'paym_' + out
                payment_methods[index] = out
            payment_methods = ', '.join(payment_methods)
        
    # Grab insurance plans s/he accepts:
    long = one_therap.find_all('div', {'class': "col-xs-12 col-sm-6 col-md-6 col-lg-6 col-tight-right"})
    if len(long) > 0:
        result = []
        for i in long:
            result.extend(i.find_all('li'))
        plans = []
        for i in range(len(result)):
            out = result[i].text
            out = re.sub(r'\([^)]*\)', '', out)
            out = out.replace(' ', '_')
            out = out.replace('|', '_')
            out = out.replace('\|', '_')
            out = out.replace('&', 'and')
            out = out.replace('-', '_')
            out = out.replace('/', '_')
            out = 'ins_' + out
            plans.append(out)
        plans = ', '.join(plans).lower()
    else:
        plans = 'xNotListed'

    # Grab specialties:
    spec_check = one_therap.find_all('li', {'class': "highlight"})
    if len(spec_check) == 0:
        specialties = 'xNotListed'
    else:
        specialties = []
        for i in range(len(spec_check)):
            out = spec_check[i].text.replace(', ', '_')
            out = out.replace(' ', '_').lower()
            out = 'spec_' + out
            specialties.append(out)
        specialties = ", ".join(specialties)
    
    # Grab issues:
    issues_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-6 col-lg-6'})
    if len(issues_check) == 0:
        issues = 'xNotListed'
    else:
        result = []
        for i in issues_check:
            result.extend(i.find_all('li'))
        issues = []
        for i in range(len(result)):
            out = result[i].text.strip()
            out = re.sub(r'\([^)]*\)', '', out)
            out = re.sub(r' $', '', out)
            out = out.replace(', ', '_')
            out = out.replace('-', '_')
            out = out.replace('\'', '')
            out = out.replace(' ', '_').lower()
            out = 'iss_' + out
            issues.append(out)
        issues = ", ".join(issues)
    
    # Grab preferred ages:
    
    # In rare cases when no specialties were selected, we need to grab the 3rd, not the 4th element:
    if specialties == '':
        age_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})[2]
    else:
        age_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})[3]
    
    age_lis = age_check.find_all('li')
    ages = []
    for i in age_lis:
        ages.append(i.text)
    
    # Creating values for age categories:
    
    if "Toddlers / Preschoolers (0 to 6)" in ages:      # Age - toddlers:
        age_toddlers = 1
    else:
        age_toddlers = 0
    
    if "Children (6 to 10)" in ages:                # Age - children:
        age_children = 1
    else:
        age_children = 0
    
    if "Preteens / Tweens (11 to 13)" in ages:      # Age - preteens:
        age_preteens = 1
    else:
        age_preteens = 0
    
    if "Adolescents / Teenagers (14 to 19)" in ages:      # Age - teens:
        age_teens = 1
    else:
        age_teens = 0
    
    if "Adults" in ages:              # Age - adults:
        age_adults = 1
    else:
        age_adults = 0
    
    if "Elders (65+)" in ages:        # Age - elders:
        age_elders = 1
    else:
        age_elders = 0
    
    # Sum of ages:
    agesum = age_toddlers+age_children+age_preteens+age_teens+age_adults+age_elders
    
    
    # Grab subcategories:
    
    if specialties == '' and agesum == 0:
        subcat_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})[2]
    elif specialties == '' and agesum > 0:
        subcat_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})[3]
    elif specialties != '' and agesum == 0:
        subcat_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})[3]
    else:
        subcat_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})[4]
    
    subcat_lis = subcat_check.find_all('li')
    subcats = []
    for i in subcat_lis:
        subcats.append(i.text)
    
    if "Aviation Professionals" in subcats:  # Subcategories - Aviation:
        sub_pilots = 1
    else:
        sub_pilots = 0
    
    if "Bisexual Clients" in subcats:  # Subcategories - HIV:
        sub_bisexuals = 1
    else:
        sub_bisexuals = 0
    
    if "Cancer" in subcats:      # Subcategories - Cancer:
        sub_cancer = 1
    else:
        sub_cancer = 0
    
    if "Gay Clients" in subcats:  # Subcategories - Gays:
        sub_gays = 1
    else:
        sub_gays = 0
    
    if "HIV / AIDS Clients" in subcats:  # Subcategories - HIV:
        sub_hiv = 1
    else:
        sub_hiv = 0
    
    if "Heterosexual Clients" in subcats:  # Subcategories - Heterosexual:
        sub_heteros = 1
    else:
        sub_heteros = 0
    
    if "Lesbian Clients" in subcats:   # Subcategories - Lesbian:
        sub_lesbians = 1
    else:
        sub_lesbians = 0
    
    if "Transgender Clients" in subcats:  # Subcategories - Transgender:
        sub_transgender = 1
    else:
        sub_transgender = 0
    
    if "Veterans" in subcats:   # Subcategories - Veterans:
        sub_veterans = 1
    else:
        sub_veterans = 0
    
    # Creating 2 additional sums
    subsum = sub_pilots+sub_bisexuals+sub_cancer+sub_gays+sub_hiv+sub_heteros+sub_lesbians+sub_transgender+sub_veterans
    age_and_sub_sum = agesum + subsum
    
    # Check if Treatment Approach is even there:
    approach_check = one_therap.find_all('h2')
    asalist = []
    count = 1
    for i in approach_check:
        try:
            out = i.text
        except:
            print("some error")
        asalist.append(out)
    if 'Treatment Approach' in asalist:
    
        # Grab treatment approaches:
        approach_check = one_therap.find_all('div', {'class': 'col-xs-12 col-sm-12 col-md-12 col-lg-12'})
        if specialties == '' and age_and_sub_sum == 0:           # if all 3 are not there
            approach_check = approach_check[2]
        elif specialties != '' and age_and_sub_sum == 0:         # if only 2 out of 3 are not there
            approach_check = approach_check[3]
        elif specialties == '' and agesum == 0 and subsum > 0:
            approach_check = approach_check[3]
        elif specialties == '' and agesum > 0 and subsum == 0:
            approach_check = approach_check[3]
        elif specialties != '' and agesum == 0  and subsum > 0:  # if only 1 out of 3 is not there
            approach_check = approach_check[4]
        elif specialties != '' and agesum > 0  and subsum == 0:
            approach_check = approach_check[4]
        elif specialties == '' and agesum > 0  and subsum > 0:
            approach_check = approach_check[4]
        else:
            approach_check = approach_check[5]

        approach_lis = approach_check.find_all('li')
        approaches = []
        for i in approach_lis:
            approaches.append(i.text)

        for i in range(len(approaches)):
            out = approaches[i]
            out = out.replace(' / ', '_')
            out = re.sub(r'\([^)]*\)', '', out)
            out = re.sub(r' $', '', out)
            out = out.strip().replace('-', '_')
            out = out.replace(' / ', '_')
            out = out.replace('/ ', '_')
            out = out.replace('/', '_')
            out = out.replace('\'', '')
            out = out.replace(' ', '_').lower()
            out = 'appr_' + out
            approaches[i] = out
        approaches = ', '.join(approaches)
    else:
        approaches = 'xNotListed'
    
    # Grab ethnicities, languages, religious orientation:
    spec_subcat = one_therap.find_all('div', {'class': "spec-subcat"})
    list_of_children = [None] * len(spec_subcat)
    for i in range(len(spec_subcat)):
        list_of_children[i] = spec_subcat[i].findChildren()
    whatsthere = []
    for i in range(len(list_of_children)):
        for z in range(len(list_of_children[i])):
            for t in [text for text in list_of_children[i][z].stripped_strings]:
                whatsthere.append(t)
    for i in range(len(whatsthere)):
        whatsthere[i] = whatsthere[i].replace(',', '')

    # Finding indexes for Ethnicity, Languages, and Religious Orientation:
    if one_therap.find_all("strong", string="Ethnicity:") == []:
        ethnicities = 'X'
        index_ethnic = ''
    else:
        index_ethnic = whatsthere.index("Ethnicity:")

    if one_therap.find_all("strong", string="Alternative Languages:") == []:
        languages = 'X'
        index_languages = ''
    else:
        index_languages = whatsthere.index("Alternative Languages:")
    index_languages
    if one_therap.find_all("strong", string="Religious Orientation:") == []:
        religion = 'X'
        index_religion = ''
    else:
        index_religion = whatsthere.index("Religious Orientation:")

    # Grabbing applicable info - depending on what's there and what's not:
    if index_ethnic != '' and index_languages != '' and index_religion != '':  # if all 3 are present
        ethnicities = whatsthere[(index_ethnic+1):index_languages]
        languages = whatsthere[(index_languages+1):index_religion]
        religion = whatsthere[(index_religion+1):len(whatsthere)]

    if index_ethnic == '' and index_languages != '' and index_religion != '':   # if only languages and religion present
        languages = whatsthere[(index_languages+1):index_religion]
        religion = whatsthere[(index_religion+1):len(whatsthere)]

    if index_ethnic != '' and index_languages == '' and index_religion != '':   # if only ethnicities and religion present
        ethnicities = whatsthere[(index_ethnic+1):index_religion]
        religion = whatsthere[(index_religion+1):len(whatsthere)]

    if index_ethnic != '' and index_languages != '' and index_religion == '':   # if only ethnicities and languages present
        ethnicities = whatsthere[(index_ethnic+1):index_languages]
        languages = whatsthere[(index_languages+1):len(whatsthere)]

    if index_ethnic != '' and index_languages == '' and index_religion != '':   # if only ethnicities and religion present
        ethnicities = whatsthere[(index_ethnic+1):index_religion]
        religion = whatsthere[(index_religion+1):len(whatsthere)]

    if index_ethnic != '' and index_languages == '' and index_religion == '':   # if only ethnicities present
        ethnicities = whatsthere[(index_ethnic+1):len(whatsthere)]

    if index_ethnic == '' and index_languages != '' and index_religion == '':   # if only languages present
        languages = whatsthere[(index_languages+1):len(whatsthere)]

    if index_ethnic == '' and index_languages == '' and index_religion != '':   # if only religion present
        religion = whatsthere[(index_religion+1):len(whatsthere)]

    ethnicities = ", ".join(ethnicities)
    languages = ", ".join(languages)
    religion = ", ".join(religion)
    
    dic['atherapist'] = [therap_name]
    dic['zipcode'] = [zipcode]
    dic['years_in_practice'] = [years_in_practice]  #.encode('utf-8', 'ignore')
    dic['school'] = [school]
    dic['yeargrad'] = [yeargrad]
    dic['board_certif'] = [board_certif]
    dic['license'] = [license]
    dic['license_state'] = [license_state]
    dic['titles'] = [titles]
    dic['per_session'] = [per_session]
    dic['per_session'] = [per_session]
    dic['sliding'] = [sliding]
    dic['insurance_yes'] = [insurance_yes]
    dic['payment_methods'] = [payment_methods]
    dic['plans'] = [plans]
    dic['specialties'] = [specialties]
    dic['issues'] = [issues]
    dic['age_toddlers'] = [age_toddlers]
    dic['age_children'] = [age_children]
    dic['age_preteens'] = [age_preteens]
    dic['age_teens'] = [age_teens]
    dic['age_adults'] = [age_adults]
    dic['age_elders'] = [age_elders]
    dic['sub_pilots'] = [sub_pilots]
    dic['sub_bisexuals'] = [sub_bisexuals]
    dic['sub_cancer'] = [sub_cancer]
    dic['sub_gays'] = [sub_gays]
    dic['sub_hiv'] = [sub_hiv]
    dic['sub_heteros'] = [sub_heteros]
    dic['sub_lesbians'] = [sub_lesbians]
    dic['sub_transgender'] = [sub_transgender]
    dic['sub_veterans'] = [sub_veterans]
    dic['approaches'] = [approaches]
    dic['client_ethnicities'] = [ethnicities]
    dic['client_languages'] = [languages]
    dic['client_religion'] = [religion]
    return pd.DataFrame(dic)

In [41]:
# Importing my two functions for webscraping:
from single_url_scrape import scrape_single_page

In [10]:
# Grabbing NYC zips. I am excluding those I've already run:    

myzips = ['10001','10002','10003','10004','10005','10006','10007','10009','10010','10011', '10012',\
          '10013','10014','10016','10017','10018','10019','10020','10021','10022','10023','10024', \
          '10025','10026','10027','10028','10029','10030','10031','10032','10033','10034','10035', \
          '10036','10037','10038','10039','10040','10044','10045','10065','10115','10119','10128', \
          '10154','10278','10280','10301','10302','10303','10304','10305','10306','10307','10309', \
          '10310','10312','10314','10451','10452','10453','10454','10455','10456','10457','10458', \
          '10459','10460','10461','10462','10463','10464','10465','10466','10467','10468','10469', \
          '10471','10472','10473','10474','10475','10514','10543','10553','10573','10701','10705', \
          '10911','10965','10977','11001','11021','11050','11101','11102','11103','11104','11105', \
          '11106','11111','11112','11201','11202','11203','11204','11205','11206','11207','11208', \
          '11209','11210','11211','11212','11213','11214','11215','11216','11217','11218','11219', \
          '11220','11221','11222','11223','11224','11225','11226','11228','11229','11230','11231', \
          '11232','11233','11234','11235','11236','11237','11238','11239','11252','11354','11355', \
          '11356','11357','11358','11360','11361','11362','11364','11365','11366','11367','11368', \
          '11369','11370','11371','11372','11373','11374','11375','11377','11378','11379','11385', \
          '11411','11412','11413','11414','11415','11416','11417','11418','11419','11420','11421', \
          '11422','11423','11424','11425','11426','11427','11428','11429','11430','11431','11432', \
          '11433','11434','11435','11436','11439','11451','11471','11510','11548','11566','11577', \
          '11580','11598','11629','11691','11692','11693','11694','11695','11731','11798','11968', \
          '12423','12428','12435','12458','12466','12473','12528','12701','12733','12734','12737', \
          '12750','12751','12754','12758','12759','12763','12764','12768','12779','12783','12786', \
          '12788','12789','13731','16091','20459']
len(myzips)

236

In [4]:
# Importing my main function for webscraping 'myscrape' saved in 'myscrape_function.py'
from myscrape_function import myscrape
# That function, in turn, imports and uses function 'scrape_single_page'
# from file 'single_url_scrape.py' that parses the BeautifulSoup of one url (one therapist)


In [6]:
myzips = ['12754', '12758']
len(myzips)

2

In [7]:
myscrape(myzips)

Starting zipcode 12754
Number of group urls for this zip is: 1
Total number of therapists in this zip is: 15
Finished url 2
Finished url 3
Finished url 4
Finished url 5
Finished url 6
Finished url 7
Finished url 9
Finished url 10
Finished url 11
Finished url 12
Finished url 13
Finished url 14
Finished url 15
Starting zipcode 12758
Number of group urls for this zip is: 1
Total number of therapists in this zip is: 20
Finished url 1
Finished url 2
Finished url 3
Finished url 4
Finished url 5
Finished url 6
Finished url 7
Finished url 8
Finished url 9
Finished url 10
Finished url 11
Finished url 12
Finished url 13
Finished url 14
Finished url 15
Finished url 16
Finished url 17
Finished url 18
Finished url 19
Finished url 20
